In [ ]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import LSTM,Dense,Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.models import Sequential
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
df=pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
df=df.dropna()

In [ ]:
y=df['label']
X=df.drop('label',axis=1)
print(X.shape)
print(y.shape)

(18285, 4)
(18285,)


In [ ]:
messages=X.copy()
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
voc_size=5000
onehot_rep=[one_hot(words,voc_size) for words in corpus ]
onehot_rep

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
embedded_docs

array([[   0,    0,    0, ..., 4668, 4829, 1978],
       [   0,    0,    0, ..., 2131,  539, 4985],
       [   0,    0,    0, ...,  122, 4009,  846],
       ...,
       [   0,    0,    0, ..., 1223,  517, 4265],
       [   0,    0,    0, ...,  839, 1888, 1412],
       [   0,    0,    0, ...,  331, 3703, 2511]])

In [ ]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4996,
       3682, 2874, 4070, 1847, 3359, 2359, 4668, 4829, 1978])

In [ ]:
X_final=np.array(embedded_docs)
y_final=np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=30,batch_size=64)

Epoch 1/30
192/192 [==============================] - 4s 12ms/step - loss: 0.3514 - accuracy: 0.8362 - val_loss: 0.2111 - val_accuracy: 0.9097
Epoch 2/30
192/192 [==============================] - 2s 10ms/step - loss: 0.1642 - accuracy: 0.9343 - val_loss: 0.1923 - val_accuracy: 0.9183
Epoch 3/30
192/192 [==============================] - 2s 10ms/step - loss: 0.1263 - accuracy: 0.9537 - val_loss: 0.2003 - val_accuracy: 0.9181
Epoch 4/30
192/192 [==============================] - 2s 10ms/step - loss: 0.1023 - accuracy: 0.9617 - val_loss: 0.2196 - val_accuracy: 0.9142
Epoch 5/30
192/192 [==============================] - 2s 10ms/step - loss: 0.0835 - accuracy: 0.9696 - val_loss: 0.2282 - val_accuracy: 0.9102
Epoch 6/30
192/192 [==============================] - 2s 10ms/step - loss: 0.0708 - accuracy: 0.9749 - val_loss: 0.2529 - val_accuracy: 0.9120
Epoch 7/30
192/192 [==============================] - 2s 9ms/step - loss: 0.0541 - accuracy: 0.9823 - val_loss: 0.2935 - val_accuracy: 0.9162


In [ ]:
y_pred=model.predict(X_test)


189/189 [==============================] - 0s 2ms/step


In [ ]:
confusion_matrix(y_test,y_pred.round())

array([[3102,  317],
       [ 219, 2397]], dtype=int64)

In [ ]:
accuracy_score(y_test,y_pred.round())

0.9111847555923778